In [ ]:
import os
import pandas as pd
import requests
from datetime import datetime, timezone
from dateutil import parser
import time

In [ ]:
from api_utils import *
import apple_health as AppleHealth
import health_connect as HealthConnect

Select whether to query the pilot study conducted in English for CAS participants or the main study in German:

## Specify project for export

In [ ]:
# PROJECT = 'PILOT_EN'
PROJECT = "MAIN_DE"

In [ ]:
load_dotenv()

# Read environment variables
private_key_path = os.getenv('RKS_PRIVATE_KEY_PATH')
service_account_name = os.getenv('RKS_SERVICE_ACCOUNT')
if PROJECT == 'PILOT_EN':
    project_id = os.getenv('RKS_PROJECT_ID_CAS')
else: 
    project_id = os.getenv('RKS_PROJECT_ID')
base_url = os.getenv('BASE_URL')
token_url = f'{base_url}/identityserver/connect/token'

In [ ]:
access_token = get_service_access_token(service_account_name, token_url)

In [ ]:
ids = get_participant_ids(access_token, base_url, project_id)
ids

Check if this worked, `ids` should contain a list of all participant IDs for this project.

In [ ]:
if PROJECT == 'PILOT_EN':
    segment_ids = {
        "iOS": "fd09bd40-a26b-42b3-86af-4a59cbba489a",
        "Android": "2c3457ae-3c5b-4616-8480-e1e4ac750cdd",
        "Garmin": "df0accf3-49ac-4436-a07c-26c2dc8a0319",
    }
else: 
    segment_ids = {
        "iOS": "d06bb52f-fecb-4625-94ee-26fddbbec8d6",
        "Android": "126ab0db-2207-47ac-afbc-f8925270c4e4"}    

## Specify data for export

Specify in the `data_specs` dictionary what kind of data and what data range you want to export.

In [ ]:
data_specs = {
    "dates": ["2025-10-07", "2025-11-30"],
    "measurements": [
        "active_calories",
        "active_calories_daily",
        "blood_glucose",
        "blood_pressure_sys",
        "blood_pressure_dia",
        "body_temp",
        "distance",
        "distance_daily",
        "exercise_segments",
        "exercise_lat",
        "exercise_lon",
        "exercise_alt",
        "exercise_hacc",
        "exercise_vacc",
        "exercise_laps",
        "exercise_time",
        "heart_rate",
        "heart_rate_min",
        "heart_rate_max",
        "oxygen_saturation",
        "respiratory_rate",
        "resting_hr",
        "sleep",
        "steps",
        "steps_daily",
        "steps_hourly",
        "steps_half_hourly",
        "total_calories",
        "total_calories_daily",
        "vo2_max",
        "weight"
        ]
}

If needed, check additionally available data types, e.g.:

In [ ]:
# HC_datatypes = HealthConnect.get_all_datatypes(access_token, project_id, base_url)
# iOS_datatypes = AppleHealth.get_all_datatypes(access_token, project_id, base_url)

In [ ]:
apple_results = None
android_results = None

## Get all AppleHealth data

In [ ]:
iOS_ids = get_participants_in_segment(access_token, base_url, project_id, segment_ids['iOS'], page_size=500)

In [ ]:
iOS_ids = get_participants_in_segment(access_token, base_url, project_id, segment_ids['iOS'], page_size=500)

apple_results = AppleHealth.fetch_measurements(
    service_access_token=access_token,
    project_id=project_id,
    ids=iOS_ids,
    base_url=base_url,
    data_specs=data_specs
)

apple_results

## Get all HealthConnect data

In [ ]:
android_ids = get_participants_in_segment(access_token, base_url, project_id, segment_ids['Android'], page_size=500)

print(android_ids)

android_results = HealthConnect.fetch_measurements(
    service_access_token=access_token,
    project_id=project_id,
    ids=android_ids,
    base_url=base_url,
    data_specs=data_specs
)

In [ ]:
def export_results(results, by="pID", export_dir="./export", data_specs=None):
    """
    Export results dict to CSVs with optional date filtering.

    Args:
        results (dict): {participantID: {measurement: DataFrame}}
        by (str): "pID" for one CSV per participant,
                  "data_type" for one CSV per measurement type
        export_dir (str): folder to save CSVs into
        data_specs (dict): may contain 'dates': ['YYYY-MM-DD', 'YYYY-MM-DD']
    """
    os.makedirs(export_dir, exist_ok=True)

    # --- Date filter setup ---
    start_date, end_date = None, None
    if data_specs and "dates" in data_specs:
        if len(data_specs["dates"]) >= 1 and data_specs["dates"][0]:
            start_date = pd.to_datetime(data_specs["dates"][0]).tz_localize("UTC")
        if len(data_specs["dates"]) >= 2 and data_specs["dates"][1]:
            # include full last day by adding 23:59:59
            end_date = pd.to_datetime(data_specs["dates"][1]).tz_localize("UTC") + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)

    def _apply_date_filter(df):
        if "observationDate" not in df.columns:
            return df
        df["observationDate"] = pd.to_datetime(df["observationDate"], errors="coerce", utc=True)
        mask = pd.Series(True, index=df.index)
        if start_date is not None:
            mask &= df["observationDate"] >= start_date
        if end_date is not None:
            mask &= df["observationDate"] <= end_date
        return df.loc[mask]

    # --- Export logic ---
    if by == "pID":
        for pid, meas_dict in results.items():
            df_list = []
            for meas, df in meas_dict.items():
                df_filtered = _apply_date_filter(df)
                if not df_filtered.empty:
                    df_list.append(df_filtered)
            if not df_list:
                continue
            merged = pd.concat(df_list, ignore_index=True)
            out_path = os.path.join(export_dir, f"{pid}.csv")
            merged.to_csv(out_path, index=False)

    elif by == "data_type":
        all_by_type = {}
        for pid, meas_dict in results.items():
            for meas, df in meas_dict.items():
                df_filtered = _apply_date_filter(df)
                if df_filtered.empty:
                    continue
                if meas not in all_by_type:
                    all_by_type[meas] = []
                all_by_type[meas].append(df_filtered)

        for meas, df_list in all_by_type.items():
            merged = pd.concat(df_list, ignore_index=True)
            out_path = os.path.join(export_dir, f"{meas}.csv")
            merged.to_csv(out_path, index=False)

    else:
        raise ValueError("Invalid value for 'by'. Use 'pID' or 'data_type'.")


Select a type of export. You can either export all data for a participant in a separate CSV file per participant (i.e., you will have all data for a person in a single file), or you can export each type of data (steps, RHR, glucose) into a separate file containing that measurement for all participants. Use the keyword `by=pID` or `by=data_type` respectively.

## Export iOS Data

In [ ]:
# Export one CSV per participant
export_results(apple_results, by="pID", export_dir="./export_participants/applehealth", data_specs=data_specs)

# # Export one CSV per measurement type
export_results(apple_results, by="data_type", export_dir="./export_datatypes/applehealth", data_specs=data_specs)


## Export Android Data

In [ ]:
# Export one CSV per participant
export_results(android_results, by="pID", export_dir="./export_participants/healthconnect", data_specs=data_specs)

# # Export one CSV per measurement type
export_results(android_results, by="data_type", export_dir="./export_datatypes/healthconnect", data_specs=data_specs)


# Check Adherence

In [ ]:
import requests
import pandas as pd
from dateutil import parser
from datetime import timedelta

def get_snack_completion(base_url, project_id, access_token, first_meal):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }

    url = f"{base_url}/api/v1/administration/projects/{project_id}/surveyanswers"
    params = {"limit": 200, "surveyName": "log_snack_de"}

    all_answers, page_id = [], None

    while True:
        if page_id:
            params["pageID"] = page_id

        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            print(f"Failed: {r.status_code}, {r.text[:200]}")
            break

        data = r.json()
        all_answers.extend(data.get("surveyAnswers", []))
        page_id = data.get("nextPageID")
        if not page_id:
            break

    print(f"[INFO] Retrieved {len(all_answers)} raw snack answers")

    if not all_answers:
        return pd.DataFrame(columns=["participantIdentifier", "date", "snacks_per_day"])

    df = pd.DataFrame(all_answers)

    # Check required fields
    if not all(col in df.columns for col in ["participantIdentifier", "surveyResultID", "date"]):
        raise KeyError("Missing one of required columns: participantIdentifier, surveyResultID, date")

    # Drop missing
    df = df.dropna(subset=["participantIdentifier", "surveyResultID", "date"])

    # Parse timestamp and extract only date (YYYY-MM-DD)
    def extract_date(x):
        try:
            return parser.isoparse(x).date()
        except Exception:
            return None

    df["date"] = df["date"].apply(extract_date)
    df = df.dropna(subset=["date"])

    # Merge with first_meal to get participant-specific start date
    merged = pd.merge(df, first_meal, on="participantIdentifier", how="left")

    # Keep only snacks within 30 days after first meal
    merged = merged[
        merged["date"] <= (merged["first_meal_date"] + pd.to_timedelta(30, unit="d"))
    ]

    # Drop duplicate surveyResultIDs
    merged_unique = merged.drop_duplicates(subset=["surveyResultID"])

    # Count snacks per participant per day
    df_snack_daily = (
        merged_unique.groupby(["participantIdentifier", "date"], as_index=False)
        .size()
        .rename(columns={"size": "snacks_per_day"})
    )

    print(f"[INFO] Produced {len(df_snack_daily)} participant-day rows")
    return df_snack_daily

In [ ]:
import requests
import pandas as pd
from dateutil import parser
from datetime import datetime, timezone, timedelta

def check_and_increment_tracking(base_url, project_id, access_token, bucket):
    headers = {"Authorization": f"Bearer {access_token}", "Accept": "application/json"}

    url = f"{base_url}/api/v1/administration/projects/{project_id}/surveytasks"
    params = {"limit": 200, "surveyName": "log_breakfast_de,log_lunch_de,log_dinner_de,log_snack_de"}

    all_tasks, page_id = [], None
    while True:
        if page_id:
            params["pageID"] = page_id
        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            print(f"Failed: {r.status_code}, {r.text[:200]}")
            break
        data = r.json()
        tasks = data.get("surveyTasks", [])
        all_tasks.extend(tasks)
        page_id = data.get("nextPageID")
        if not page_id:
            break

    print(f"[INFO] Retrieved {len(all_tasks)} total tasks")

    if not all_tasks:
        return pd.DataFrame(columns=[
            "participantIdentifier", "meal_completed", "meal_total", "meal_pct",
            "days_2_plus_incl_snacks", "days_total", "days_2_plus_incl_snacks_pct"
        ])

    df = pd.DataFrame(all_tasks)[["participantIdentifier", "surveyName", "status", "endDate"]]

    # Robust datetime parsing — no .dt accessor used
    def safe_parse_date(x):
        try:
            return parser.parse(x).date()
        except Exception:
            return None

    df["date"] = df["endDate"].apply(safe_parse_date)

    meal_names = {"log_breakfast_de", "log_lunch_de", "log_dinner_de"}

    # ---- MEAL COUNTS ----
    df_meals = df[df["surveyName"].isin(meal_names)]
    meal_total = df_meals.groupby("participantIdentifier").size().reset_index(name="meal_total")
    meal_completed = (
        df_meals[df_meals["status"].str.lower() == "complete"]
        .groupby("participantIdentifier")
        .size()
        .reset_index(name="meal_completed")
    )

    df_meal_summary = pd.merge(meal_total, meal_completed, on="participantIdentifier", how="outer").fillna(0)
    df_meal_summary["meal_total"] = df_meal_summary["meal_total"].astype(int)
    df_meal_summary["meal_completed"] = df_meal_summary["meal_completed"].astype(int)
    df_meal_summary["meal_pct"] = (
        (df_meal_summary["meal_completed"] / df_meal_summary["meal_total"] * 100)
        .round(1)
        .fillna(0)
    )

    # ---- COMPLETED SURVEYS (MEAL + SNACK) ----
    df_complete = df[df["status"].str.lower() == "complete"]

    df_day_counts = (
        df_complete.groupby(["participantIdentifier", "date"])
        .size()
        .reset_index(name="count_per_day")
    )

    days_2plus = (
        df_day_counts[df_day_counts["count_per_day"] >= 2]
        .groupby("participantIdentifier")
        .size()
        .reset_index(name="days_2_plus_incl_snacks")
    )

    # ---- TOTAL DAY RANGE (first meal → +30 days or today) ----
    today = datetime.now(timezone.utc).date()
    df_meals_nonan = df_meals.dropna(subset=["date"])

    first_meal = (
        df_meals_nonan.groupby("participantIdentifier", as_index=False)["date"]
        .min()
        .rename(columns={"date": "first_meal_date"})
    )
    
    first_meal["days_total"] = first_meal["first_meal_date"].apply(
        lambda d: max(0, (min(d + timedelta(days=30), today) - d).days + 1)
    )

    df_days = pd.merge(first_meal, days_2plus, on="participantIdentifier", how="outer").fillna(0)
    df_days["days_2_plus_incl_snacks"] = df_days["days_2_plus_incl_snacks"].astype(int)
    df_days["days_total"] = df_days["days_total"].astype(int)
    df_days["days_2_plus_incl_snacks_pct"] = (
        (df_days["days_2_plus_incl_snacks"] / df_days["days_total"] * 100)
        .round(1)
        .fillna(0)
    )

    # ---- SNACK COUNTS ----
    df_snacks = get_snack_completion(base_url, project_id, access_token, first_meal)
    snacks_total = (
        df_snacks.groupby("participantIdentifier")
        .size()
        .reset_index(name="snacks_total")
    )

    # ---- FINAL MERGE ----
    df_final = (
        df_meal_summary
        .merge(snacks_total, on="participantIdentifier", how="outer")
        .merge(df_days, on="participantIdentifier", how="outer")
        .fillna(0)
    )

    df_final["snacks_total"] = df_final["snacks_total"].astype(int)

    df_final = df_final[[
        "participantIdentifier",
        "meal_completed", "meal_total", "meal_pct",
        "snacks_total",
        "days_2_plus_incl_snacks", "days_total", "days_2_plus_incl_snacks_pct"
    ]]
    
    return df_final


In [ ]:
import requests
import pandas as pd
from dateutil import parser
from datetime import datetime, timezone, timedelta

def check_and_increment_tracking(base_url, project_id, access_token, bucket):
    headers = {"Authorization": f"Bearer {access_token}", "Accept": "application/json"}

    url = f"{base_url}/api/v1/administration/projects/{project_id}/surveytasks"
    params = {"limit": 200, "surveyName": "log_breakfast_de,log_lunch_de,log_dinner_de,log_snack_de"}

    all_tasks, page_id = [], None
    while True:
        if page_id:
            params["pageID"] = page_id
        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            print(f"Failed: {r.status_code}, {r.text[:200]}")
            break
        data = r.json()
        tasks = data.get("surveyTasks", [])
        all_tasks.extend(tasks)
        page_id = data.get("nextPageID")
        if not page_id:
            break

    print(f"[INFO] Retrieved {len(all_tasks)} total tasks")

    if not all_tasks:
        return pd.DataFrame(columns=[
            "participantIdentifier", "meal_completed", "meal_total", "meal_pct",
            "days_2_plus_incl_snacks", "days_total", "days_2_plus_incl_snacks_pct"
        ])

    df = pd.DataFrame(all_tasks)[["participantIdentifier", "surveyName", "status", "endDate"]]

    def safe_parse_date(x):
        try:
            return parser.parse(x).date()
        except Exception:
            return None

    df["date"] = df["endDate"].apply(safe_parse_date)

    meal_names = {"log_breakfast_de", "log_lunch_de", "log_dinner_de"}

    # ---- MEAL COUNTS (overall) ----
    df_meals = df[df["surveyName"].isin(meal_names)]
    meal_total = df_meals.groupby("participantIdentifier").size().reset_index(name="meal_total")
    meal_completed = (
        df_meals[df_meals["status"].str.lower() == "complete"]
        .groupby("participantIdentifier")
        .size()
        .reset_index(name="meal_completed")
    )

    df_meal_summary = pd.merge(meal_total, meal_completed, on="participantIdentifier", how="outer").fillna(0)
    df_meal_summary["meal_total"] = df_meal_summary["meal_total"].astype(int)
    df_meal_summary["meal_completed"] = df_meal_summary["meal_completed"].astype(int)
    df_meal_summary["meal_pct"] = (
        (df_meal_summary["meal_completed"] / df_meal_summary["meal_total"] * 100)
        .round(1)
        .fillna(0)
    )

    # ---- COMPLETED SURVEYS (meal + snack) ----
    df_complete = df[df["status"].str.lower() == "complete"]

    df_day_counts = (
        df_complete.groupby(["participantIdentifier", "date"])
        .size()
        .reset_index(name="count_per_day")
    )

    days_2plus = (
        df_day_counts[df_day_counts["count_per_day"] >= 2]
        .groupby("participantIdentifier")
        .size()
        .reset_index(name="days_2_plus_incl_snacks")
    )

    # ---- TOTAL DAY RANGE ----
    today = datetime.now(timezone.utc).date()
    df_meals_nonan = df_meals.dropna(subset=["date"])

    first_meal = (
        df_meals_nonan.groupby("participantIdentifier", as_index=False)["date"]
        .min()
        .rename(columns={"date": "first_meal_date"})
    )
    first_meal["days_total"] = first_meal["first_meal_date"].apply(
        lambda d: max(0, (min(d + timedelta(days=30), today) - d).days + 1)
    )

    # ---- LIMIT TO FIRST 30 DAYS PER PARTICIPANT ----
    df = pd.merge(df, first_meal[["participantIdentifier", "first_meal_date"]], on="participantIdentifier", how="left")
    df["date"] = pd.to_datetime(df["date"])
    df["first_meal_date"] = pd.to_datetime(df["first_meal_date"])
    df["date_diff"] = (df["date"] - df["first_meal_date"]).dt.days
    df = df[df["date_diff"].between(0, 30)]  # keep only days 0–30

    # ---- DAYS SUMMARY ----
    df_complete = df[df["status"].str.lower() == "complete"]
    df_day_counts = (
        df_complete.groupby(["participantIdentifier", "date"])
        .size()
        .reset_index(name="count_per_day")
    )
    days_2plus = (
        df_day_counts[df_day_counts["count_per_day"] >= 2]
        .groupby("participantIdentifier")
        .size()
        .reset_index(name="days_2_plus_incl_snacks")
    )

    df_days = pd.merge(first_meal, days_2plus, on="participantIdentifier", how="outer").fillna(0)
    df_days["days_2_plus_incl_snacks"] = df_days["days_2_plus_incl_snacks"].astype(int)
    df_days["days_total"] = df_days["days_total"].astype(int)
    df_days["days_2_plus_incl_snacks_pct"] = (
        (df_days["days_2_plus_incl_snacks"] / df_days["days_total"] * 100)
        .round(1)
        .fillna(0)
    )

    # ---- SNACK COUNTS ----
    df_snacks_daily = get_snack_completion(base_url, project_id, access_token, first_meal)

    # ---- MEAL COUNTS (daily) ----
    df_meals_completed = df_meals[df_meals["status"].str.lower() == "complete"]
    df_meals_daily = (
        df_meals_completed.groupby(["participantIdentifier", "date"], as_index=False)
        .size()
        .rename(columns={"size": "meal_completed_day"})
    )

    # ---- MERGE DAILY TABLES ----
    df_daily = pd.merge(
        df_meals_daily,
        df_snacks_daily,
        on=["participantIdentifier", "date"],
        how="outer"
    ).fillna(0)

    df_daily["meal_completed_day"] = df_daily["meal_completed_day"].astype(int)
    df_daily["snacks_per_day"] = df_daily["snacks_per_day"].astype(int)

    # ---- OVERALL SUMMARY TABLE ----
    snacks_total = (
        df_snacks_daily.groupby("participantIdentifier")
        .size()
        .reset_index(name="snacks_total")
    )

    df_final = (
        df_meal_summary
        .merge(snacks_total, on="participantIdentifier", how="outer")
        .merge(df_days, on="participantIdentifier", how="outer")
        .fillna(0)
    )

    df_final["snacks_total"] = df_final["snacks_total"].astype(int)

    df_final = df_final[[
        "participantIdentifier",
        "meal_completed", "meal_total", "meal_pct",
        "snacks_total",
        "days_2_plus_incl_snacks", "days_total", "days_2_plus_incl_snacks_pct"
    ]]
    
    print(f"[INFO] Daily meal/snack table rows: {len(df_daily)}")
    return df_final, df_daily


In [ ]:
import requests
import pandas as pd
from dateutil import parser
from datetime import datetime, timezone, timedelta

def check_and_increment_tracking(base_url, project_id, access_token, bucket):
    headers = {"Authorization": f"Bearer {access_token}", "Accept": "application/json"}

    url = f"{base_url}/api/v1/administration/projects/{project_id}/surveytasks"
    params = {"limit": 200, "surveyName": "log_breakfast_de,log_lunch_de,log_dinner_de,log_snack_de"}

    all_tasks, page_id = [], None
    while True:
        if page_id:
            params["pageID"] = page_id
        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            print(f"Failed: {r.status_code}, {r.text[:200]}")
            break
        data = r.json()
        tasks = data.get("surveyTasks", [])
        all_tasks.extend(tasks)
        page_id = data.get("nextPageID")
        if not page_id:
            break

    print(f"[INFO] Retrieved {len(all_tasks)} total tasks")

    if not all_tasks:
        return pd.DataFrame(columns=[
            "participantIdentifier", "meal_completed", "meal_total", "meal_pct",
            "days_2_plus_incl_snacks", "days_total", "days_2_plus_incl_snacks_pct"
        ])

    # ---- INITIAL CLEANING ----
    print(pd.DataFrame(all_tasks).columns)
    df = pd.DataFrame(all_tasks)[["participantIdentifier", "surveyName", "status", "insertedDate"]]
    # print(df.head(30))
    print(f"Task status options: {df['status'].unique()}")

    def safe_parse_date(x):
        try:
            return parser.parse(x).date()
        except Exception:
            return None

    df["date"] = df["insertedDate"].apply(safe_parse_date)

    # ---- DETERMINE FIRST MEAL DATE ----
    meal_names = {"log_breakfast_de", "log_lunch_de", "log_dinner_de"}
    df_meals_initial = df[df["surveyName"].isin(meal_names)]
    df_meals_nonan = df_meals_initial.dropna(subset=["date"])

    first_meal = (
        df_meals_nonan.groupby("participantIdentifier", as_index=False)["date"]
        .min()
        .rename(columns={"date": "first_meal_date"})
    )

    # ---- LIMIT TO FIRST 30 DAYS PER PARTICIPANT ----
    df = pd.merge(df, first_meal, on="participantIdentifier", how="left")
    df["date"] = pd.to_datetime(df["date"])
    df["first_meal_date"] = pd.to_datetime(df["first_meal_date"])
    df["date_diff"] = (df["date"] - df["first_meal_date"]).dt.days
    df = df[df["date_diff"].between(0, 30)]  # inclusive: first + 29 = 30 days total

    # ---- DAILY STATUS COUNTS ----
    df_status_daily = (
        df[df["status"].isin(["complete", "incomplete", "closed"])]
        .assign(status=lambda x: x["status"].replace({"closed": "incomplete"}))
        .groupby(["participantIdentifier", "date", "status"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
        .rename(columns={
            "complete": "complete_meals",
            "incomplete": "incomplete_meals"
        })
    )

    # ensure both columns exist even if one status is missing
    for col in ["complete_meals", "incomplete_meals"]:
        if col not in df_status_daily.columns:
            df_status_daily[col] = 0

    # print(df_status_daily.head(30))

    # --- GET SNACK COUNTS PER DAY ----
    df_snacks_daily = get_snack_completion(base_url, project_id, access_token, first_meal)

    # ensure consistent datetime dtype and normalize (strip time)
    df_status_daily["date"] = pd.to_datetime(df_status_daily["date"]).dt.date
    df_snacks_daily["date"] = pd.to_datetime(df_snacks_daily["date"]).dt.date

    # merge on both participant and date
    df_final = (
        df_status_daily
        .merge(
            df_snacks_daily,
            on=["participantIdentifier", "date"],
            how="outer"
        )
        .fillna(0)
    )

    # ---- DAILY DERIVED COUNTS ----
    df_final["meals_total_day"] = df_final["complete_meals"] + df_final["incomplete_meals"]
    df_final["has_2plus_complete_meals"] = df_final["complete_meals"] >= 2
    df_final["has_2plus_any"] = (df_final["complete_meals"] + df_final["snacks_per_day"]) >= 2

    # ---- AGGREGATE PER PARTICIPANT ----
    df_summary = (
        df_final.groupby("participantIdentifier")
        .agg(
            meals_completed=("complete_meals", "sum"),
            meals_total=("meals_total_day", "sum"),
            nb_days=("date", "nunique"),
            nb_days_2plus_meals=("has_2plus_complete_meals", "sum"),
            nb_days_2plus_any=("has_2plus_any", "sum")
        )
        .reset_index()
    )

    # ---- DERIVED PERCENTAGES ----
    df_summary["percentage_meals_tracked"] = (
        df_summary["meals_completed"] / df_summary["meals_total"] * 100
    ).round(1)

    df_summary["pct_days_2plus_meals"] = (
        df_summary["nb_days_2plus_meals"] / (df_summary["nb_days"]-1) * 100
    ).round(1)

    df_summary["pct_days_2plus_any"] = (
        df_summary["nb_days_2plus_any"] / (df_summary["nb_days"]-1) * 100
    ).round(1)

    return df_summary


In [ ]:
df_summary = check_and_increment_tracking(base_url, project_id, access_token, None)

In [ ]:
df_summary

- `meals_completed` ... number of main meal entries completed over the duration of the study
- `meals_total` ... total number of meal surveys delivered to the participant
- `nb_days` ... number of days accounted for (31 includes the day of enrollment where only one sample survey is sent)
- `nb_days_2plus_meals` ... number of days where the participant completed at least 2 main meal surveys
- `nb_days_2plus_any` ... number of days where the participant completed at least 2 main meals OR snack entries
- `percentage_meals_tracked` ... percentage of main meals tracked
- `pct_days_2plus_meals` ... percentage of days on which 2+ main meals were tracked (excl. enrollment day)
- `pct_days_2plus_any` ... percentage of days on which 2+ main meals OR snacks were tracked (excl. enrollment day) - **basis for participant payment**